## Service Functions

### Get Litmus Circuit

In [ ]:
def get_litmus_circuit(qubits_count, circuit_name=None, registers_count=1):
    
    quantum_registers = []
    
    for register_index in range(registers_count):
        
        qubits_per_register = qubits_count // registers_count
        
        register_name = f"{circuit_name}_{register_index}"
    
        quantum_register = qiskit.QuantumRegister(qubits_per_register,
                                                  register_name)
        
        quantum_registers.append(quantum_register)
    
    litmus_circuit = qiskit.QuantumCircuit(*quantum_registers)
    
    qubits = list(range(qubits_count))
    
    # Parameterized RZs

    for qubit in qubits:
        
        parameter = qiskit.circuit.Parameter(f"{circuit_name}_{qubit}")

        litmus_circuit.rz(parameter, qubit)
        
    # CX Ladder

    for control_qubit, target_qubit in zip(qubits, qubits[1:]):

        litmus_circuit.cx(control_qubit, target_qubit)
        
    # Last-to-first CX

    litmus_circuit.cx(qubits[-1], qubits[0])  

    litmus_circuit.barrier(label=circuit_name)

    return litmus_circuit

### Get CNOT Circuit

In [ ]:
def get_cnot_circuit(qubits_count, circuit_name=None, cnot_qubits=None, registers_count=1):
    
    if cnot_qubits is None:
        
        cnot_qubits = [0, 1]
    
    quantum_registers = []
    
    for register_index in range(registers_count):
        
        qubits_per_register = qubits_count // registers_count
        
        register_name = f"{circuit_name}_{register_index}"
    
        quantum_register = qiskit.QuantumRegister(qubits_per_register,
                                                  register_name)
        
        quantum_registers.append(quantum_register)
    
    cnot_circuit = qiskit.QuantumCircuit(*quantum_registers)
    
    qubits = list(range(qubits_count))
    
    # Parameterized RZs

    for qubit in qubits:
        
        parameter = qiskit.circuit.Parameter(f"{circuit_name}_{qubit}")

        cnot_circuit.rz(parameter, qubit)
        
       
    # CNOT

    cnot_circuit.cx(*cnot_qubits)  

    cnot_circuit.barrier(label=circuit_name)

    return cnot_circuit

### Get Sinusoids

In [ ]:
def get_sinusoids(qubits_count,
                  frequencies_count=None, 
                  frequencies=None, amplitudes=None,
                  min_amplitude=1, max_amplitude=3):
    
    samples_count = 2 ** qubits_count
    
    time_samples = np.linspace(0, 2 * np.pi, samples_count)
    
    if frequencies is None:
    
        frequencies = np.random.uniform(low=0, 
                                        high=samples_count // 2, 
                                        size=(frequencies_count,)).astype(int)
    if amplitudes is None:
        
        frequencies_count = len(frequencies)
        
        amplitudes = np.random.uniform(low=min_amplitude, 
                                       high=max_amplitude, 
                                       size=(frequencies_count,))
    
    sinusoids = np.zeros(samples_count)
    
    for amplitude, frequency in zip(amplitudes, frequencies):
        
        sinusoid = amplitude * np.sin(frequency * time_samples)
        
        sinusoids = sinusoids + sinusoid
        
    normalized_sinusoids = sinusoids / np.sqrt(np.sum(sinusoids ** 2))
    
    # print("frequencies:", frequencies)
    # print("amplitudes:", amplitudes)
    # print("sinusoids:", sinusoids)
    # print("normalized_sinusoids:", normalized_sinusoids)
    
    return normalized_sinusoids

### Get IBM Cost

In [ ]:
def get_ibm_cost(qiskit_circuit,
                 depth_penalty_factor=0.995,
                 one_qubit_gate_fidelity=0.9996,
                 two_qubit_gate_fidelity=0.99):

    # Input Parameters - https://arxiv.org/abs/2202.14025
    
    # https://github.com/ArlineQ/arline_quantum/blob/8fa31271f4882c4c1a92ff500a86daa03a9cfae2/arline_quantum/hardware/ibm.py
    
    remove_barriers_pass = qiskit.transpiler.passes.RemoveBarriers()
    
    circuit = remove_barriers_pass(qiskit_circuit)
    
    circuit.remove_final_measurements()
    
    depth = circuit.depth()
    
    # Fidelities
    
    fidelities_dict = {1: one_qubit_gate_fidelity,
                       2: two_qubit_gate_fidelity}
    
    fidelities = []
    
    operation_counts = circuit.count_ops()

    for operation_name, operation_count in operation_counts.items():

        # qubit_count = qubit_count_per_gate[operation_name]
        
        first_instruction = circuit.get_instructions(operation_name)[0]
        
        qubit_count = first_instruction.operation.num_qubits

        gate_fidelity = fidelities_dict[qubit_count]

        fidelity = gate_fidelity ** operation_count

        fidelities.append(fidelity)

        # print("  operation_name:", operation_name)
        # print("  gate_fidelity:", gate_fidelity)
        # print("  operation_count:", operation_count)
        # print("  fidelity:", fidelity)
        # print()
        
    # IBM Cost - https://arxiv.org/abs/2008.08571

    ibm_cost = depth_penalty_factor ** depth * np.prod(fidelities)

    return ibm_cost

## Get Full Map

In [ ]:
def get_full_map(transpiled_circuit, printout=False):
    
    # No Layout
    
    if transpiled_circuit.layout is None:
        
        full_map = list(range(transpiled_circuit.num_qubits))
        
        return full_map
    
    # Zero Map
    
    input_qubit_mapping = transpiled_circuit.layout.input_qubit_mapping
    initial_layout = transpiled_circuit.layout.initial_layout
    final_layout = transpiled_circuit.layout.final_layout
 
    zero_map = dict(zip(input_qubit_mapping,
                        transpiled_circuit.qubits))

    # After Layout Map

    after_layout_map = [()] * transpiled_circuit.num_qubits
    
    for physical, in_virtual in initial_layout.get_physical_bits().items():

        out_virtual = zero_map[in_virtual]

        after_layout_map[physical] = (in_virtual, out_virtual)
    
    
    # After Routing Map
    
    after_routing_map = after_layout_map.copy()
    
    if final_layout is not None:
        
        for from_row, qubit in enumerate(transpiled_circuit.qubits):
            
            to_row = final_layout[qubit]
            
            from_in, from_out = after_layout_map[from_row]
            to_in, to_out = after_layout_map[to_row]
            
            after_routing_map[to_row] = (to_in, from_out)
            
    # Full Map
    
    full_map = []
    
    for out_qubit in transpiled_circuit.qubits:
    
        for physical, virtuals in enumerate(after_routing_map):
            
            in_virtual, out_virtual = virtuals

            if out_virtual is out_qubit:

                full_map.append(physical)
                
    # Printout
    
    if printout is True:
        
        display("zero_map:", zero_map)
        display("after_layout_map:", after_layout_map)
        display("after_routing_map:", after_routing_map)
        display("full_map:", full_map)
        display("transpiled_circuit.layout:", transpiled_circuit.layout)

    return full_map

## Transpile Functions

### Transpile

In [ ]:
@functools.wraps(qiskit.transpile)
def transpile(*arguments, **key_arguments):

    transpiled_circuit, transpiler_options = transpile_and_return_options(*arguments, **key_arguments)
    
    return transpiled_circuit

### Transpile and Return Options

In [ ]:
def transpile_and_return_options(circuit, backend=None, *arguments, **key_arguments):
    
    # Options
    
    transpiler_options = key_arguments.get('transpiler_options', dict())
    
    options_backend = transpiler_options.get('backend')
    options_arguments = transpiler_options.get('arguments', ())

    
    # Arguments
    
    run_backend = backend or options_backend
    
    run_arguments = arguments or options_arguments
    
    run_key_arguments = {**transpiler_options, **key_arguments}
    
    run_key_arguments['backend'] = run_backend
    
    
    # Dynamical Decoupling Arguments
    
    dd_pulses = run_key_arguments.pop('dd_pulses', None)
    dd_pulses_count = run_key_arguments.pop('dd_pulses_count', None)
    dd_pulse_alignment = run_key_arguments.pop('dd_pulse_alignment', None)
    dynamical_decoupling = run_key_arguments.pop('dynamical_decoupling', None)

    
    # Stack
    
    stack_pass_manager = None
    
    arguments_stack = key_arguments.get('stack')    
    options_stack = transpiler_options.get('stack')
    
    stack = arguments_stack or options_stack
    
    if stack is not None:
        
        stack_pass_manager = get_stack_pass_manager(**run_key_arguments)
        
        
    # Pass Manager
    
    arguments_pass_manager = key_arguments.get('pass_manager')
    options_pass_manager = transpiler_options.get('pass_manager')
    
    pass_manager = arguments_pass_manager or options_pass_manager or stack_pass_manager
    
    
    # Transpile
    
    if pass_manager is None:
        
        transpiled_circuit = qiskit.transpile(circuit, *run_arguments, **run_key_arguments)
        
    else:
        transpiled_circuit = pass_manager.run(circuit)
        
        
    # Dynamical Decoupling
    
    if dynamical_decoupling is True:
        
        transpiled_circuit = add_dynamical_decoupling(transpiled_circuit, run_backend,
                                                      dd_pulses, dd_pulses_count, dd_pulse_alignment)

    # Transpiler Options
    
    options = run_key_arguments.copy()
    
    full_map = get_full_map(transpiled_circuit)
    
    options['full_map'] = full_map
    options['arguments'] = run_arguments
    options['original_circuit'] = circuit
    options['transpiled_circuit'] = transpiled_circuit
    

    return transpiled_circuit, options

### Transpile Chain

In [ ]:
def transpile_chain(circuits, backend=None, *arguments, **key_arguments):

    qubits_count = getattr(backend, 'num_qubits',
                           backend.configuration().num_qubits)

    chain_circuit = qiskit.QuantumCircuit(qubits_count)
    
    qubits = list(range(qubits_count))
    
    full_map = None
    
    # Chain
    
    for circuit in circuits:
        
        if full_map is not None:
        
            initial_layout = full_map[:circuit.num_qubits]

            key_arguments['initial_layout'] = initial_layout
        
        transpiled_circuit, transpiler_options = transpile_and_return_options(
            circuit,
            backend,
            *arguments, **key_arguments)
        
        chain_circuit.compose(
            transpiled_circuit,
            qubits=qubits,
            inplace=True
        )
        
        full_map = transpiler_options['full_map']
        
    return chain_circuit

### Transpile Right

In [ ]:
def transpile_right(central_circuit, right_circuit,
                    backend=None, *arguments, **key_arguments):
    
    full_map = get_full_map(central_circuit)

    key_arguments['initial_layout'] = full_map[:right_circuit.num_qubits]
    
    transpiled_right_circuit, transpiler_options = transpile_and_return_options(
        right_circuit,
        backend,
        *arguments, **key_arguments
    )
    
    resulting_circuit = central_circuit.compose(transpiled_right_circuit)
    
    # No Layout
    
    if transpiled_right_circuit.layout is None:
        
        return resulting_circuit
    
    
    # Central Routing
    
    central_routing = list(range(central_circuit.num_qubits))
        
    if central_circuit.layout.final_layout is not None:
        
        central_routing = [central_circuit.layout.final_layout[qubit]
                           for qubit in central_circuit.qubits]
    
    # Right Routing

    right_routing = central_routing.copy()
    
    if transpiled_right_circuit.layout.final_layout is not None:  
    
        right_routing = [transpiled_right_circuit.layout.final_layout[qubit] 
                         for qubit in transpiled_right_circuit.qubits]
        
    # Final Routing
    
    final_routing = [right_routing[qubit] for qubit in central_routing]   
    
    # Layouts
    
    final_layout = qiskit.transpiler.Layout.from_intlist(final_routing, *resulting_circuit.qregs)
    
    transpile_layout = qiskit.transpiler.TranspileLayout(
        input_qubit_mapping=central_circuit.layout.input_qubit_mapping,
        initial_layout=central_circuit.layout.initial_layout,
        final_layout=final_layout
    )

    resulting_circuit._layout = transpile_layout
    
    # Printouts
    
    # display("central_routing:", central_routing)
    # display("right_routing:", right_routing)
    # display("final_routing:", final_routing)
    # display("final_layout:", final_layout)
    
    return resulting_circuit

### Transpile Left

In [ ]:
def transpile_left(central_circuit, left_circuit,
                   backend=None, *arguments, **key_arguments):
    
    # Initial Map
    
    initial_layout = central_circuit.layout.initial_layout
    input_qubit_mapping = central_circuit.layout.input_qubit_mapping

    initial_map = [initial_layout[qubit] for qubit in input_qubit_mapping]
    
    key_arguments['initial_layout'] = initial_map[:left_circuit.num_qubits]
    
    
    # Transpile
    
    inverted_left_circuit = left_circuit.inverse()
    
    transpiled_inverted_left_circuit, transpiler_options = transpile_and_return_options(
        inverted_left_circuit,
        backend,
        *arguments, **key_arguments
    )
    
    transpiled_left_circuit = transpiled_inverted_left_circuit.inverse()
    
    transpiled_left_circuit._layout = transpiled_inverted_left_circuit.layout

    resulting_circuit = central_circuit.compose(transpiled_left_circuit,
                                                front=True)
    # No Layout
    
    if transpiled_left_circuit.layout is None:
        
        return resulting_circuit
    
    
    # Left Routing
    
    left_routing = list(range(transpiled_left_circuit.num_qubits))

    if transpiled_left_circuit.layout.final_layout is not None:
        
        left_routing = [transpiled_left_circuit.layout.final_layout[qubit]
                        for qubit in transpiled_left_circuit.qubits]
    
    # Central Routing
    
    central_routing = left_routing.copy()
    
    if central_circuit.layout.final_layout is not None:
    
        central_routing = [central_circuit.layout.final_layout[qubit] 
                           for qubit in central_circuit.qubits]
        
    # Final Routing
    
    final_routing = [central_routing[qubit] for qubit in left_routing]   
    
    # Final Layout
    
    final_layout = qiskit.transpiler.Layout.from_intlist(final_routing, *resulting_circuit.qregs)
    
    
    # Initial Layout
    
    input_qubit_mapping = transpiled_left_circuit.layout.input_qubit_mapping
    
    initial_map = get_full_map(transpiled_left_circuit)

    initial_layout = transpiled_left_circuit.layout.initial_layout.copy()

    for virtual, physical in zip(input_qubit_mapping, initial_map):

        initial_layout[virtual] = physical


    # Transpile Layout
    
    transpile_layout = qiskit.transpiler.TranspileLayout(
        input_qubit_mapping=input_qubit_mapping,
        initial_layout=initial_layout,
        final_layout=final_layout
    )

    resulting_circuit._layout = transpile_layout
    
    # Printouts
    
    # display("left_routing:", left_routing)
    # display("central_routing:", central_routing)
    # display("final_routing:", final_routing)
    # display("final_layout:", final_layout)
    
    return resulting_circuit

## Stacks

### Imports

In [ ]:
import bqskit
from bqskit.ext import bqskit_to_qiskit

import pytket
import pytket.extensions.qiskit

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

### Get Stack Pass Manager

In [ ]:
def get_stack_pass_manager(stack="qiskit", **key_arguments):
    
    backend = key_arguments.pop("backend", None)
    qsearch_block_size = key_arguments.pop("qsearch_block_size", None)
    
    pass_manager = generate_preset_pass_manager(**key_arguments)
    
    
    # Stacks

    if stack == "qiskit_qsearch":
        
        pass_manager.init.append([
            QSearchPass(backend=backend,
                        qsearch_block_size=qsearch_block_size)])
        
    if stack == "qiskit_qfactor_qsearch":
        
        pass_manager.init.append([
            QFactorPass(backend=backend,
                        qsearch_block_size=qsearch_block_size)])
        
    if stack == "qiskit_pytket":
        
        ibmq_backend = key_arguments.get('backend')
        
        noise_model = None
        
        if ibmq_backend is not None:
              
            noise_model = ibmq_backend.options.noise_model
        
        pytket_ibmq_backend = pytket.extensions.qiskit.AerBackend(
            noise_model=noise_model)

        pass_manager.init.append([
            PytketPass(pytket_backend=pytket_ibmq_backend)])    
        
    return pass_manager   

### Model from IBMQ Backend

In [5]:
def model_from_ibmq_backend(ibmq_backend):
    
    if isinstance(ibmq_backend, (qiskit.providers.BackendV1,
                                 qiskit.providers.fake_provider.FakeBackend)):
        
        # print('IBMQ Backend Version 1')
        
        qubits_count = ibmq_backend.configuration().n_qubits
        basis_gates = ibmq_backend.configuration().basis_gates
        coupling_map = ibmq_backend.configuration().coupling_map
        
    if isinstance(ibmq_backend, (qiskit.providers.BackendV2,
                                 qiskit.providers.fake_provider.FakeBackendV2)):
        
        # print('IBMQ Backend Version 2')
        
        qubits_count = ibmq_backend.target.num_qubits
        basis_gates = ibmq_backend.target.operation_names
        coupling_map = ibmq_backend.target.build_coupling_map()
        
        
    coupling_list = list({tuple(sorted(edge)) for edge in coupling_map})
    
    gate_dict = {'cx': bqskit.ir.gates.CNOTGate(),
                 'cz': bqskit.ir.gates.CZGate(),
                 'u3': bqskit.ir.gates.U3Gate(),
                 'u2': bqskit.ir.gates.U2Gate(),
                 'u1': bqskit.ir.gates.U1Gate(),
                 'rz': bqskit.ir.gates.RZGate(),
                 'sx': bqskit.ir.gates.SXGate(),
                 'x': bqskit.ir.gates.XGate(),
                 'p': bqskit.ir.gates.RZGate()}
    
    if len(basis_gates) > 10:
        
        gate_set = {bqskit.ir.gates.CNOTGate(), 
                    bqskit.ir.gates.RZGate(), 
                    bqskit.ir.gates.SXGate()}
    else:        
        
        gate_set = {gate_dict.get(basis_gate) 
                    for basis_gate in basis_gates} - {None}
        
    machine_model = bqskit.MachineModel(qubits_count, coupling_list, gate_set)
    
    return machine_model

### QSearch

In [31]:
def run_qsearch_synthesis(bqskit_circuit, block_size):

    # compilation_task = bqskit.compiler.CompilationTask(bqskit_circuit,
    #                                                    [bqskit.passes.QSearchSynthesisPass()])

    compilation_task = bqskit.compiler.CompilationTask(bqskit_circuit, [
        bqskit.passes.QuickPartitioner(block_size=block_size),
        bqskit.passes.ForEachBlockPass([
            bqskit.passes.QSearchSynthesisPass(),
            bqskit.passes.ScanningGateRemovalPass()
        ]),
        bqskit.passes.UnfoldPass()
    ])

    with bqskit.compiler.Compiler() as compiler:
        synthesized_circuit = compiler.compile(compilation_task)

    return synthesized_circuit


class QSearchPass(qiskit.transpiler.basepasses.TransformationPass):

    def __init__(self, backend, qsearch_block_size=None):
        
        if qsearch_block_size is None:
            qsearch_block_size = 2
        
        super().__init__()
        
        self.machine_model = model_from_ibmq_backend(backend)
        self.qsearch_block_size = qsearch_block_size


    def run(self, dag):
        
        # print("Running QSearchPass")
                
        qiskit_circuit = qiskit.converters.dag_to_circuit(dag)
        
        bqskit_circuit = bqskit.ext.qiskit_to_bqskit(qiskit_circuit)
        
        synthesized_circuit = run_qsearch_synthesis(bqskit_circuit, self.qsearch_block_size)        

        transpiled_qiskit_circuit = bqskit.ext.bqskit_to_qiskit(synthesized_circuit)
        
        new_dag = qiskit.converters.circuit_to_dag(transpiled_qiskit_circuit)

        return new_dag

### QFactor

In [32]:
class QFactorPass(qiskit.transpiler.basepasses.TransformationPass):

    def __init__(self, backend, qsearch_block_size=None):
        
        if qsearch_block_size is None:
            qsearch_block_size = 2
        
        super().__init__()
        
        self.machine_model = model_from_ibmq_backend(backend)
        self.qsearch_block_size = qsearch_block_size


    def run(self, dag):
        
        # print("Running QFactorPass")
                
        qiskit_circuit = qiskit.converters.dag_to_circuit(dag)
        
        bqskit_circuit = bqskit.ext.qiskit_to_bqskit(qiskit_circuit)
        
        # Ansatz
        
        qubits_count = bqskit_circuit.num_qudits
        
        qubit_pairs = np.transpose(np.triu_indices(qubits_count, 1))
        
        ansatz_circuit = bqskit.ir.circuit.Circuit(qubits_count)
        
        for qubit_pair in qubit_pairs:

            ansatz_circuit.append_gate(bqskit.ir.gates.VariableUnitaryGate(num_qudits=2),
                                       location=qubit_pair)
            
        # QFactor Optimization
        
        target = bqskit_circuit.get_unitary()
        
        instantiated_circuit = ansatz_circuit.copy()

        instantiated_circuit.instantiate(
            target=target,
            method='qfactor',
            diff_tol_a=1e-12,   # Stopping criteria for distance change
            diff_tol_r=1e-6,    # Relative criteria for distance change
            dist_tol=1e-12,     # Stopping criteria for distance
            max_iters=100000,   # Maximum number of iterations
            min_iters=1000,     # Minimum number of iterations
            slowdown_factor=0,  # Larger numbers slowdown optimization to avoid local minima
        )
        
        # QSearch Synthesis
        
        synthesized_circuit = run_qsearch_synthesis(instantiated_circuit, self.qsearch_block_size)        

        transpiled_qiskit_circuit = bqskit.ext.bqskit_to_qiskit(synthesized_circuit)
        
        new_dag = qiskit.converters.circuit_to_dag(transpiled_qiskit_circuit)

        return new_dag

### Pytket

In [33]:
class PytketPass(qiskit.transpiler.basepasses.TransformationPass):
    
    # https://github.com/CQCL/tket/blob/7553dc7d384604ef26a00a9aa29c7d58517cec3b/pytket/pytket/backends/backend.py#L147

    def __init__(self, pytket_backend):
        
        super().__init__()
        
        self.pytket_backend = pytket_backend


    def run(self, dag):
        
        # print("Running PytketPass")
                
        qiskit_circuit = qiskit.converters.dag_to_circuit(dag)
        
        pytket_circuit = pytket.extensions.qiskit.qiskit_to_tk(qiskit_circuit)
        
        
        # Compilation
        
        compilation_pass = self.pytket_backend.default_compilation_pass(optimisation_level=2)
        
        compilation_pass.apply(pytket_circuit)
        
        
        # Rebase
        
        IBMQ_GATES = {pytket.OpType.CX,
                      pytket.OpType.Rz,
                      pytket.OpType.SX,
                      pytket.OpType.X}
                
        rebase_pass = pytket.passes.SequencePass([
            pytket.passes.FullPeepholeOptimise(),
            pytket.passes.auto_rebase_pass(gateset=IBMQ_GATES)
        ])

        rebase_pass.apply(pytket_circuit)
        

        transpiled_qiskit_circuit = pytket.extensions.qiskit.tk_to_qiskit(pytket_circuit)
        
        new_dag = qiskit.converters.circuit_to_dag(transpiled_qiskit_circuit)

        return new_dag

### Add Dynamical Decoupling

In [ ]:
def add_dynamical_decoupling(circuit, backend, 
                             dd_pulses, dd_pulses_count, dd_pulse_alignment):
    
    DEFAULT_DD_PULSES = [qiskit.circuit.library.XGate()]
    DEFAULT_DD_PULSES_COUNT = 2
    
    # DD Sequence
    
    if dd_pulses is None:
        dd_pulses = DEFAULT_DD_PULSES
        
    if dd_pulses_count is None:
        dd_pulses_count = DEFAULT_DD_PULSES_COUNT
    
    dd_sequence = dd_pulses * dd_pulses_count
    
    
    # Pulse Alignment
    
    backend_pulse_alignment = None

    if (hasattr(backend, 'configuration') and
        hasattr(backend.configuration(), 'timing_constraints') and
        hasattr(backend.configuration().timing_constraints, 'pulse_alignment')):

        backend_pulse_alignment = backend.configuration().timing_constraints.get('pulse_alignment')
        
    run_pulse_alignment = dd_pulse_alignment or backend_pulse_alignment
    
    
    # Instruction Durations
    
    instruction_durations = qiskit.transpiler.InstructionDurations.from_backend(backend)
    
    
    # DD Pass Manager
        
    dd_pass_manager = qiskit.transpiler.PassManager([
        qiskit.transpiler.passes.ALAPScheduleAnalysis(instruction_durations),
        qiskit.transpiler.passes.PadDynamicalDecoupling(
            durations=instruction_durations,
            dd_sequence=dd_sequence, 
            pulse_alignment=run_pulse_alignment)])
    
    circuit_with_dd = dd_pass_manager.run(circuit)
        
    return circuit_with_dd